In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [2]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
import LinearAlgebra: ldiv!, dot
using Test
using ForwardDiff
using SpecialFunctions
using BenchmarkTools
using TransportMap
using QuadGK
using Polynomials
using Distributions
using Random
using LoopVectorization
using Optim
using NLsolve
using MLDataUtils
using MLDataPattern
using Test
using SparseArrays
using QRupdate
using AdaptiveTransportMap: vander, transform!, evaluate, ncoeff, optimize, negative_log_likelihood!, derivative

# using Profile
# using ProfileView

┌ Info: Precompiling AdaptiveTransportMap [bdf749b0-1400-4207-80d3-e689c0e3f03d]
└ @ Base loading.jl:1278
  ** incremental compilation may be fatally broken for this module **

┌ Warning: Type annotations on keyword arguments not currently supported in recipes. Type information has been discarded
└ @ RecipesBase ~/.julia/packages/RecipesBase/AN696/src/RecipesBase.jl:117


### 1D example

In [3]:
Nx = 3
Ne = 500
X = randn(Nx, Ne)
ens = EnsembleState(X)
m = 10
B = MultiBasis(CstProHermite(3), Nx)

idx = [0 0 0; 2 0 1; 0 1 0; 0 2 1; 0 1 2; 1 0 0; 2 2 2]

coeff =  randn(size(idx,1))

C = MapComponent(m, Nx, idx, coeff)
S = Storage(C.I.f, X);

In [8]:
@time G = grad_x(C.I, X);

  0.000624 seconds (281 allocations: 1.725 MiB)


In [9]:
ForwardDiff.gradient(x->begin
                           x[end] = 0.0
                           return  C.I.f.f(x)
        end, member(ens,1)) 

3-element Array{Float64,1}:
 -0.16541992175819617
  0.631479634052268
  0.0

In [10]:
G[1,:]

3-element Array{Float64,1}:
 -0.1654199217581962
  0.6314796340522678
  0.0

In [7]:
quadgk(t->ForwardDiff.gradient(x->C.I.g(ForwardDiff.gradient(y->C.I.f.f(y), vcat(x[1:end-1],t))[end]), member(ens,1)), 0, x[end])[1]

LoadError: UndefVarError: x not defined

In [502]:
ForwardDiff.gradient(x->C.I.g(ForwardDiff.gradient(y->C.I.f.f(y), x)[end]), member(ens,1))

3-element Array{Float64,1}:
 -0.0031958783853288566
 -0.019712542299756518
  0.07999422617176781

In [ ]:
    ψt[i] = R.f.f(vcat(x[1:end-1], 0.0)) + quadgk(t->R.g(ForwardDiff.gradient(y->R.f.f(y), vcat(x[1:end-1],t))[end]), 0, x[end])[1]


In [476]:
@time evaluate(C.I, member(ens,1))

  0.000095 seconds (195 allocations: 21.172 KiB)


-0.8628070969383215

In [443]:
ψbasis = zeros(Ne, C.I.Nψ, C.I.Nx-1)
for i=1:Nx-1
    ψbasis_i = view(ψbasis, :, :, i)    
    ψbasis_i .= evaluate_basis(C.I.f.f, X, [i], C.I.f.f.idx)
end

dxψbasis = zero(ψbasis)
fill!(dxψbasis, 1.0)
@inbounds for i=1:Nx-1
    for j=1:Nx-1
        if i==j
        dxψbasis[:,:,i] .*= grad_xk_basis(C.I.f.f, X, 1, j, j, C.I.f.f.idx)       
        else
        dxψbasis[:,:,i] .*= ψbasis[:,:,j]
        end
    end
end

In [444]:
dxψbasis

500×7×2 Array{Float64,3}:
[:, :, 1] =
 0.0   0.0706838  0.0   0.0  0.0   0.268701     0.031373
 0.0   0.619948   0.0  -0.0  0.0  -0.0493918   -0.334673
 0.0   0.540452   0.0  -0.0  0.0  -0.134541    -0.0584185
 0.0   0.467153   0.0   0.0  0.0   0.176139     0.213213
 0.0   0.392058   0.0  -0.0  0.0   0.206614    -0.0499594
 0.0   0.399866   0.0   0.0  0.0   0.203846     0.160889
 0.0   0.255391   0.0  -0.0  0.0   0.244027    -0.0856339
 0.0  -0.113459   0.0  -0.0  0.0  -0.264098     0.0606189
 0.0   0.563771   0.0   0.0  0.0  -0.116968     0.176689
 0.0   0.589929   0.0  -0.0  0.0  -0.092469    -0.210954
 0.0   0.274027   0.0   0.0  0.0  -0.239999     0.147535
 0.0   0.284554   0.0  -0.0  0.0   0.237588    -0.0379391
 0.0   0.63152    0.0   0.0  0.0   0.00455436   0.128664
 ⋮                                 ⋮           
 0.0   0.54643    0.0  -0.0  0.0   0.130316    -0.00520103
 0.0   0.591372   0.0   0.0  0.0   0.0908967    0.300248
 0.0   0.619876   0.0   0.0  0.0   0.0495423    0.21

In [451]:
NxX, Ne = size(X)

out = zero(X')
Nx = C.I.Nx
x0 = zeros(Ne)
xlast = view(X,Nx,:)
ψk0  = repeated_evaluate_basis(C.I.f.f, x0)
dxkψk0 = repeated_grad_xk_basis(C.I.f.f, x0)
ψoff = evaluate_offdiagbasis(C.I.f, X)
dxkψ = zero(ψoff)

coeff = C.I.f.f.coeff
cache = zeros((Nx-1)*Ne)
cacheg = zeros(Ne)

ψbasis = evaluate_basis(C.I.f.f, X)

function integrand!(v::Vector{Float64}, t::Float64)
    dxkψ .= repeated_grad_xk_basis(C.I.f.f,  t*xk)
    @avx @. cacheg = (dxkψ * ψoff) *ˡ coeff
    evaluate!(cacheg, C.I.g, cacheg)
    
    @inbounds for i=1:Nx-1
        vi = view(v, (i-1)*Ne+1:i*Ne)
        dxψbasisi = view(dxψbasis,:,:,i)
        @avx @. vi = ((dxψbasisi * dxkψ) *ˡ coeff) * cacheg
    end
end

@time quadgk!(integrand!, cache, 0.0, 1.0; rtol = 1e-3)
# Compute

lastcol = view(out, :, Nx)
@time evaluate!(lastcol, C.I.g, (repeated_grad_xk_basis(C.I.f.f,  xlast) .* ψoff)*coeff)

@avx @. lastcol += (ψoff * dxkψk0) *ˡ coeff

  0.315810 seconds (477.77 k allocations: 22.649 MiB)
  0.000045 seconds (26 allocations: 84.375 KiB)


500-element view(::Array{Float64,2}, :, 3) with eltype Float64:
 0.7751924865968259
 0.9800160866751729
 1.1553360465789302
 0.9090923564324922
 1.3548474979561214
 0.9407246101979496
 1.5309473684190684
 0.6616688870629657
 1.2365400443315697
 1.1299514583577188
 1.6281852662633376
 1.372743025798409
 1.2452628381533983
 ⋮
 1.1760017733047508
 0.9851005143125123
 1.187098789668456
 1.0427627739806737
 1.0146358695255762
 1.1107423708442055
 1.142885219013601
 1.0132047900994348
 1.412455498465029
 1.6464952204574939
 1.1542217823383096
 1.3044711279180727

In [407]:
ψoff = evaluate_offdiagbasis(C.I.f, X)

500×7 Array{Float64,2}:
 1.0   0.479024    0.631123  -0.0353804  0.631123  0.503884  -0.016948
 1.0   0.516113    0.374151  -0.541487   0.374151  0.462891  -0.279468
 1.0   0.541685    0.426964  -0.534361   0.426964  0.378012  -0.289455
 1.0   0.489067    0.473316   0.508477   0.473316  0.494696   0.248679
 1.0   0.481665    0.404763   0.540012   0.404763  0.501563   0.260105
 1.0   0.0505771   0.565151  -0.376906   0.565151  0.630604  -0.0190628
 1.0   0.00692681  0.291534   0.512679   0.291534  0.6316     0.00355123
 1.0  -0.197895    0.626753  -0.110237   0.626753  0.615505   0.0218155
 1.0  -0.209152    0.621276  -0.159663   0.621276  0.613532   0.033394
 1.0   0.014198    0.243043  -0.475035   0.243043  0.631539  -0.00674453
 1.0   0.227063    0.495265  -0.488476   0.495265  0.610123  -0.110915
 1.0  -0.54158     0.630418   0.0549984  0.630418  0.37571   -0.029786
 1.0   0.537241    0.55703    0.394929   0.55703   0.417586   0.212172
 ⋮                                             

500×7 Array{Float64,2}:
 0.0   0.127516    0.0  -0.00941824  -0.0603125  0.0   0.00161962
 0.0   0.0435432   0.0  -0.0456839    0.223395   0.0  -0.166863
 0.0   0.0201696   0.0  -0.0198969    0.266855   0.0  -0.180912
 0.0  -0.123671    0.0  -0.128579     0.0986575  0.0   0.0518345
 0.0   0.022335    0.0   0.0250406    0.251496   0.0   0.161614
 0.0  -0.0125383   0.0   0.0934366    0.133367   0.0  -0.00449851
 0.0  -0.00182364  0.0  -0.134975     0.039055   0.0   0.000475736
 0.0   0.0506894   0.0   0.0282365    0.117903   0.0   0.00410388
 0.0  -0.056042    0.0  -0.0427815    0.0510777  0.0   0.00274546
 0.0   0.00371959  0.0  -0.12445      0.035293   0.0  -0.000979394
 0.0   0.0494312   0.0  -0.10634      0.177455   0.0  -0.0397412
 0.0   0.103301    0.0  -0.0104904    0.273578   0.0  -0.012926
 0.0   0.135817    0.0   0.09984      0.116324   0.0   0.0443079
 ⋮                                               ⋮    
 0.0   0.101323    0.0  -0.042474    -0.157166   0.0   0.0230094
 0.0  -

In [383]:
grad_xk_basis(C.I.f.f, X, 1)

LoadError: MethodError: no method matching grad_xk_basis(::ExpandedFunction, ::Array{Float64,2})
Closest candidates are:
  grad_xk_basis(::ExpandedFunction, ::Array{Float64,2}, !Matched::Int64, !Matched::Union{Int64, Array{Int64,1}}) at /media/mat/HDD/AdaptiveTransportMap/src/mapcomponent/expandedfunction.jl:212
  grad_xk_basis(::ExpandedFunction, ::Array{Float64,2}, !Matched::Int64, !Matched::Union{Int64, Array{Int64,1}}, !Matched::Union{UnitRange{Int64}, Int64, Array{Int64,1}}) at /media/mat/HDD/AdaptiveTransportMap/src/mapcomponent/expandedfunction.jl:206
  grad_xk_basis(::ExpandedFunction, ::Array{Float64,2}, !Matched::Int64, !Matched::Union{Int64, Array{Int64,1}}, !Matched::Union{UnitRange{Int64}, Int64, Array{Int64,1}}, !Matched::Array{Int64,2}) at /media/mat/HDD/AdaptiveTransportMap/src/mapcomponent/expandedfunction.jl:203
  ...

In [379]:
S.cache_dψxd

500-element Array{Float64,1}:
 0.5315090572833349
 0.533730559598309
 0.541909995665149
 0.5355811539493861
 0.5571281699625658
 0.5565316120379733
 0.5124949977493904
 0.5555240856533903
 0.5423587968679733
 0.5096760537539106
 0.5395453266572324
 0.5653728367538514
 0.5602706051863584
 ⋮
 0.5042187428462308
 0.5706072756934936
 0.5343514622216646
 0.544895253625552
 0.5682763811862693
 0.5325873519410239
 0.4892322362035903
 0.5339410251209131
 0.5578817861094277
 0.5661470400907131
 0.5625440400892824
 0.5727060698190826

In [377]:
negative_log_likelihood!(J, dJ, coeff, S, C, X)

1.5497820652192922

In [378]:
dJ

7-element Array{Float64,1}:
  0.45669780258552556
  0.01609914141985005
  0.22549906081776191
  0.015452904377808637
  0.03749560693255646
  0.2449869376256962
 -0.00045954427350969963

### Test Hessian of the log π(x)

In [30]:
Nx = 2
Ne = 8
m = 10

X = randn(Nx, Ne)

idx = [0 0; 0 1; 1 0; 1 1; 1 2]
coeff =   [0.6285037650645056;
 -0.4744029092496623;
  1.1405280011620331;
 -0.7217760771894809;
  0.11855056306742319];

C = MapComponent(m, Nx, idx, coeff);

In [34]:
dψ_basis = grad_x_basis(C.I.f.f, X)

8×5×2 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0  -0.207296    -0.130929     0.00117735
 0.0  0.0  -0.216523    -0.134105     0.0375507
 0.0  0.0   0.00618292   0.00360034   0.00205305
 0.0  0.0  -0.199714    -0.0830991   -0.107373
 0.0  0.0   0.209274     0.0991538   -0.106331
 0.0  0.0  -0.236677    -0.10036     -0.126702
 0.0  0.0  -0.270316    -0.120917     0.142049
 0.0  0.0  -0.0785179   -0.0495593    0.00259814

[:, :, 2] =
 0.0   0.00283979  0.0   0.00155384  0.345581
 0.0   0.0867129   0.0   0.0465448   0.31942
 0.0  -0.166026    0.0  -0.104855    0.307967
 0.0  -0.268816    0.0  -0.149159    0.0381479
 0.0   0.254048    0.0   0.138466    0.109751
 0.0  -0.26767     0.0  -0.064575    0.0207737
 0.0   0.262746    0.0   0.0959916   0.0506544
 0.0   0.0165449   0.0   0.0102847   0.391819

In [41]:
dxlogDJ = zeros(Ne, Nx)

ψoffdψxd = grad_xd(C.I.f.f, X)

8-element Array{Float64,1}:
  0.03850011681624252
 -0.036864339594153354
  0.19095459274793916
  0.23970874743411022
 -0.20745155095311177
  0.17605463803346075
 -0.18792702339659972
  0.03117818085903034

In [50]:
grad_x_grad_xd(C.I.f.f, X, C.I.f.f.idx)

8×2 Array{Float64,2}:
 -0.0150962    0.275081
 -0.00172344   0.272766
  0.00109835   0.191537
 -0.0403773   -0.00821251
 -0.0333779    0.127178
 -0.0481415    0.0110568
  0.0468207    0.0774556
 -0.00492957   0.294571

In [51]:
hess_xd(C.I.f.f, X)

8-element Array{Float64,1}:
  0.27508054698884865
  0.2727658334063561
  0.19153741860252596
 -0.008212514607585927
  0.12717812747627785
  0.01105679171823458
  0.07745559049870646
  0.29457111787580165

In [52]:
dxdxkψx_t = ForwardDiff.hessian(C.I.f.f, X[:,1])

2×2 Array{Float64,2}:
 -0.13341    -0.0150962
 -0.0150962   0.275081

In [ ]:
evaluate()

In [16]:
log_pdf(C, X)

8-element Array{Float64,1}:
 -1.890933371333304
 -1.0774421758804786
 -1.3812358207093065
 -2.756566903253647
 -2.2764943782683207
 -3.268114225886461
 -0.9741216255311081
 -1.442529469931936

In [18]:
Cx = evaluate(C, X)

8-element Array{Float64,1}:
 -1.3182090265735815
  0.5518744832820461
  1.0207836492619973
  1.969508900840876
  1.7095646000932772
  2.2193925767103297
  0.06731649217667446
  1.0856697228586412

In [22]:
getidx(C)

5×2 Array{Int64,2}:
 0  0
 0  1
 1  0
 1  1
 1  2

In [21]:
getcoeff(C)

5-element Array{Float64,1}:
  0.6285037650645056
 -0.4744029092496623
  1.1405280011620331
 -0.7217760771894809
  0.11855056306742319

In [23]:
dψ = grad_x_basis(C.I.f.f, X)

8×5×2 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0   0.107478    0.0190908  -0.043005
 0.0  0.0   0.0865389   0.0541193  -0.0107887
 0.0  0.0   0.138011    0.0855902   0.0231549
 0.0  0.0  -0.270698   -0.113793   -0.145219
 0.0  0.0  -0.259926   -0.128072   -0.127645
 0.0  0.0  -0.225594   -0.0869868  -0.122217
 0.0  0.0  -0.266312   -0.155186    0.0881003
 0.0  0.0   0.261878    0.157944    0.0678763

[:, :, 2] =
 0.0   0.200064   0.0   0.122532   -0.167233
 0.0   0.0623342  0.0   0.0386104   0.379666
 0.0  -0.0838875  0.0  -0.0502452   0.357862
 0.0  -0.268231   0.0  -0.105483    0.0306983
 0.0  -0.24554    0.0  -0.112246    0.113373
 0.0  -0.270878   0.0  -0.142266    0.00262786
 0.0   0.165408   0.0   0.0714071   0.211025
 0.0  -0.129595   0.0  -0.0583847   0.246624

In [24]:
d2ψ = hess_x_basis(C.I.f.f, X)

8×5×2×2 Array{Float64,4}:
[:, :, 1, 1] =
 0.0  0.0  -0.287372   -0.0510446    0.114986
 0.0  0.0  -0.297614   -0.18612      0.0371031
 0.0  0.0  -0.267679   -0.166006    -0.0449099
 0.0  0.0  -0.0102897  -0.00432547  -0.00552002
 0.0  0.0  -0.0807765  -0.0398007   -0.0396678
 0.0  0.0  -0.1657     -0.0638922   -0.089769
 0.0  0.0  -0.0515665  -0.030049     0.0170591
 0.0  0.0  -0.0730321  -0.0440471   -0.0189292

[:, :, 2, 1] =
 0.0   0.0   0.0   0.0215025   -0.0293467
 0.0   0.0   0.0   0.00539434   0.0530439
 0.0  -0.0   0.0  -0.0115774    0.0824581
 0.0  -0.0  -0.0   0.0726096   -0.0211313
 0.0  -0.0  -0.0   0.0638224   -0.0644633
 0.0  -0.0  -0.0   0.0611086   -0.00112877
 0.0   0.0  -0.0  -0.0440502   -0.130179
 0.0  -0.0   0.0  -0.0339381    0.143359

[:, :, 1, 2] =
 0.0   0.0   0.0   0.0215025   -0.0293467
 0.0   0.0   0.0   0.00539434   0.0530439
 0.0  -0.0   0.0  -0.0115774    0.0824581
 0.0  -0.0  -0.0   0.0726096   -0.0211313
 0.0  -0.0  -0.0   0.0638224   -0.0644633
 0.0  -

### Integration

In [20]:
Nx = 100
Ny = 50
m = 20
Ne = 400
X = randn(Nx, Ne).*randn(Nx, Ne) #.+ 0.5*randn(Nx).*cos.(randn(Nx, Ne) .* randn(Nx, Ne))

# L = LinearTransform(X)
# transform!(L, X);
# C = MapComponent(m, Nx)
# @time C, _ = optimize(C, X, 10)

In [24]:
M = HermiteMap(m, X);

In [25]:
@time M_noqr = optimize(M, X, 10; P = serial, start = Ny+1);

  9.550136 seconds (42.61 M allocations: 13.653 GiB, 7.59% gc time)


In [26]:
@time M_qr = optimize(M, X, 10; P = serial, withqr = true, start = Ny+1);

 18.997304 seconds (45.81 M allocations: 21.111 GiB, 4.17% gc time)


In [31]:
function timing()
    M = HermiteMap(m, X)
    @btime begin 
        optimize($M, $X, $10; P = serial, withqr = false, start = Ny+1)
    end
    
    @btime begin 
        optimize($M, $X, $10; P = serial, withqr = true, start = Ny+1)
    end
end

timing (generic function with 1 method)

In [32]:
timing();

  11.257 s (42606958 allocations: 13.65 GiB)
  21.346 s (45813192 allocations: 21.12 GiB)


In [11]:
timing();

  14.597 s (42433511 allocations: 13.37 GiB)


In [29]:
timing();

  973.450 ms (1054425 allocations: 1.43 GiB)


In [14]:
methods(update_component)

# 1 method for generic function "update_component":
[1] update_component(C::MapComponent, X, reduced_margin::Array{Int64,2}, S::Storage) in AdaptiveTransportMap at /media/mat/HDD/AdaptiveTransportMap/src/mapcomponent/greedyfit.jl:249

In [30]:
function timing()
    @btime begin 
        M = HermiteMap($m, $X)
        optimize($M, $X, $10; P = thread)
    end
end

timing (generic function with 1 method)

In [88]:
timing();

  297.730 ms (1000725 allocations: 1.35 GiB)


### Test optimization with QR

In [45]:
Nx = 3
Ne = 8
m = 20

idx = [0 0 0; 0 0 1; 0 1 0; 0 1 1; 0 1 2; 1 0 0]


Nψ = 6
coeff = [ 0.20649582065364197;
         -0.5150990160472986;
          2.630096893080717;
          1.13653076177397;
          0.6725837371023421;
         -1.3126095306624133]
C = MapComponent(m, Nx, idx, coeff; α = 1e-6);

Ne = 100


# The QR decomposition is not unique!

X = randn(Nx, Ne) .* randn(Nx, Ne) + cos.(randn(Nx, Ne)) .* exp.(-randn(Nx, Ne).^2)

L = LinearTransform(X)
transform!(L, X)
S = Storage(C.I.f, X)
F = QRscaling(S)
newidx = [1 1 1]

Snew = update_storage(S, X, newidx)
Fupdated = updateQRscaling(F, Snew)

Fnew = QRscaling(Snew)

@test norm(Fupdated.D - Fnew.D)<1e-8
@test norm(Fupdated.Dinv - Fnew.Dinv)<1e-8

Test Passed